In [1]:
#tweepy package to coolect tweets
import tweepy
import twitter_keys

#authentication
auth = tweepy.OAuthHandler(twitter_keys.CONSUMER_KEY, twitter_keys.CONSUMER_SECRET)
auth.set_access_token(twitter_keys.ACCESS_KEY, twitter_keys.ACCESS_SECRET)

#create instance of api
api = tweepy.API(auth , wait_on_rate_limit=True)
print("logged in")

logged in


In [2]:
import os
import pandas as pd
import numpy as np
import datetime
from textblob import TextBlob
import re

#ENTER SEARCH TERM HERE
search = "python"

#enter the max number of tweets
max_tweets = 3000

#arrays to collect data of only non rt tweets
timestamps = list()
tweets = list()
tweets_retweets = list()
tweets_likes = list()
tweets_length = list()
tweets_source = list()
tweets_sentiment = list()

#arrays to collect data of all tweets
all_tweets_timestamps = list();
all_tweets_tweets = list()
all_tweets_retweets = list()
all_tweets_likes = list()
all_tweets_length = list()
all_tweets_source = list()

#cursor to scrape data from twitter
for tweet in tweepy.Cursor(api.search,
                            q= search,
                            count = 100 ,
                            result_type="recent",
                            include_entities=True,
                            lang="en" ,
                            tweet_mode="extended").items(max_tweets):
    
    all_tweets_timestamps.append(tweet.created_at)
    all_tweets_tweets.append(tweet.full_text)
    all_tweets_retweets.append(tweet.retweet_count)
    all_tweets_likes.append(tweet.favorite_count)
    all_tweets_length.append(len(tweet.full_text))
    all_tweets_source.append(tweet.source)

    #only load tweet to arrays if the NOT retweet
    if(tweet.full_text[0:2] != "RT"):
        
        #analyze the sentiment
        temp = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.full_text).split())
        analysis = TextBlob(temp)
        if analysis.sentiment.polarity < 0:
            num = -1
        elif analysis.sentiment.polarity > 0:
            num = 1
        else:
            num = 0
         
        #add sentiment to array
        tweets_sentiment.append(num)
        
        #add all attributes of tweet to arrays
        timestamps.append(tweet.created_at)
        tweets.append(tweet.full_text)
        tweets_retweets.append(tweet.retweet_count)
        tweets_likes.append(tweet.favorite_count)
        tweets_length.append(len(tweet.full_text))
        tweets_source.append(tweet.source)

#message when done
print("done querying.")

now = datetime.datetime.now()
datestr = now.strftime('%Y-%m-%d')

 
#check if the directory exist
#if it doesn't create it
directory = "data/" + datestr + "/"

if not os.path.exists(directory):
    os.makedirs(directory)
    
directory = "data/" + datestr + "/" + search + "/"

if not os.path.exists(directory):
    os.makedirs(directory)
    
file = directory + search + "_data.csv"

#use pandas package to export data to csv file
#prepare dataframe to export only on rt tweets to csv
df = pd.DataFrame({'timestamp':timestamps, 
                   'likes': tweets_likes , 
                   'retweets': tweets_retweets , 
                   "source" : tweets_source ,
                   "length" : tweets_length ,
                   "sentiment" : tweets_sentiment ,
                   'tweet':tweets})

#create csv
df.to_csv(file , sep=',', encoding='utf-8')

#output to user
print("non RT file created: " , file)

file = directory + "all_tweets_" + search + "_data.csv"

#prepare a dataframe to export to csv of all tweets
df = pd.DataFrame({'timestamp':all_tweets_timestamps , 
                   'likes': all_tweets_likes , 
                   'retweets': all_tweets_retweets , 
                   "source" : all_tweets_source ,
                   "length" : all_tweets_length ,
                   'tweet':all_tweets_tweets})

#create csv
df.to_csv(file , sep=',', encoding='utf-8')

#output to user
print("full file created: " , file)

#create metadata and output to csv file
meta_df = pd.DataFrame({"all_retweets" : [np.mean(all_tweets_retweets)] , 
                        "all_likes" : [np.mean(all_tweets_likes)] , 
                        "all_length" : [np.mean(all_tweets_length)] ,
                        "retweets" : [np.mean(tweets_retweets)] , 
                        "likes" : [np.mean(tweets_likes)] ,
                        "length" : [np.mean(tweets_length)] ,
                        "sentiment" : [np.mean(tweets_sentiment)] , 
                        "tweet-rt-ratio" : [len(tweets_retweets) / len(all_tweets_retweets)]})

file = directory + "metadata_" + search + "_data.csv"
#create csv
meta_df.to_csv(file , sep=',', encoding='utf-8')

done querying.
non RT file created:  data/2018-11-02/python/python_data.csv
full file created:  data/2018-11-02/python/all_tweets_python_data.csv


In [3]:
import numpy as np

sources = np.array(all_tweets_source)
unique, counts = np.unique(sources, return_counts=True)

df = pd.DataFrame({"source" : unique , "count" : counts , "percentage" : (counts / sum(counts)) * 100})
df

,source,count,percentage
0,30days30sites,10,0.333333
1,5b33c38ba60ea948624684,1,0.033333
2,@100DaysOfCode_,5,0.166667
3,A new way forward,1,0.033333
4,Adcourier,1,0.033333
5,Aldo Zaffalon,1,0.033333
6,Auto-Blogger,2,0.066667
7,BlARROW123,4,0.133333
8,Bot Libre!,17,0.566667
9,BubbleLife,1,0.033333


In [4]:
df.max()

source        viewandhue-xyz
count                    701
percentage           23.3667
dtype: object